## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, PolynomialFeatures,OneHotEncoder,LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,RidgeCV,LassoCV,ElasticNetCV
from sklearn import metrics


## Import Data

In [2]:
#load train_cleaned.csv
train = pd.read_csv('../datasets/train_cleaned.csv')
train.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,...,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice,age_garage
0,60,RL,69.0552,13517,Pave,None,IR1,Lvl,AllPub,CulDSac,...,0,None,None,None,0,3,2010,WD,130500,34.0
1,60,RL,43.0000,11492,Pave,None,IR1,Lvl,AllPub,CulDSac,...,0,None,None,None,0,4,2009,WD,220000,12.0
2,20,RL,68.0000,7922,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,1,2010,WD,109000,57.0
3,60,RL,73.0000,9802,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,4,2010,WD,174000,3.0
4,50,RL,82.0000,14235,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,3,2010,WD,138500,53.0


In [3]:
test = pd.read_csv('../datasets/test_cleaned.csv')
test.head()

,id,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,age_garage
0,2658,190,RM,69.0000,9142,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,None,None,None,0,4,2006,WD,96.0
1,2718,90,RL,69.0552,9662,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,None,0,8,2006,WD,29.0
2,2414,60,RL,58.0000,17104,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,None,0,9,2006,New,0.0
3,1989,30,RM,60.0000,8520,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,None,0,7,2007,WD,72.0
4,625,20,RL,69.0552,9500,Pave,None,IR1,Lvl,AllPub,...,185,0,None,None,None,0,7,2009,WD,46.0


## Pre-processing

### Get_dummies

In [4]:
# ms_subclas to categorical type
train['ms_subclass'] = train.ms_subclass.astype('str')
test['ms_subclass'] = test.ms_subclass.astype('str')

In [5]:
#create categorical_columns / numerical_columns
categorical_columns = [col for col in train.select_dtypes('object').columns]
numerical_columns = [col for col in train.columns if col not in categorical_columns]

In [6]:
train = pd.get_dummies(train,columns=categorical_columns,drop_first=True)
train.head()

,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,...,misc_feature_Shed,misc_feature_TenC,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,69.0552,13517,6,8,1976,2005,289.0,533.0,0.0,192.0,...,0,0,0,0,0,0,0,0,0,1
1,43.0000,11492,7,5,1996,1997,132.0,637.0,0.0,276.0,...,0,0,0,0,0,0,0,0,0,1
2,68.0000,7922,5,7,1953,2007,0.0,731.0,0.0,326.0,...,0,0,0,0,0,0,0,0,0,1
3,73.0000,9802,5,5,2006,2007,0.0,0.0,0.0,384.0,...,0,0,0,0,0,0,0,0,0,1
4,82.0000,14235,6,8,1900,1993,0.0,0.0,0.0,676.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
train.shape

(2045, 274)

In [8]:
test = pd.get_dummies(test,columns=categorical_columns,drop_first=True)
test.head()

,id,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,...,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
0,2658,69.0000,9142,6,8,1910,1950,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2718,69.0552,9662,5,4,1977,1977,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2414,58.0000,17104,7,5,2006,2006,0.0,554,0,...,0,0,0,0,0,0,1,0,0,0
3,1989,60.0000,8520,5,6,1923,2006,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,625,69.0552,9500,6,5,1963,1963,247.0,609,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
col_to_add = [col for col in train.columns if (col not in test.columns)&(col !='saleprice')]
len(col_to_add)

26

In [10]:
col_to_add  # These are the columns that are in train data
            # if these are selected as features, when apply to test data it will generate error

['ms_subclass_150',
 'ms_zoning_C (all)',
 'utilities_NoSeWa',
 'neighborhood_GrnHill',
 'neighborhood_Landmrk',
 'condition_2_Feedr',
 'condition_2_RRAe',
 'condition_2_RRAn',
 'condition_2_RRNn',
 'roof_matl_Membran',
 'exterior_1st_CBlock',
 'exterior_1st_ImStucc',
 'exterior_1st_Stone',
 'exterior_2nd_Stone',
 'bsmt_cond_Fa',
 'bsmt_cond_Po',
 'heating_OthW',
 'heating_Wall',
 'heating_qc_Po',
 'electrical_Mix',
 'functional_Sal',
 'functional_Sev',
 'garage_qual_Fa',
 'pool_qc_Fa',
 'pool_qc_Gd',
 'misc_feature_TenC']

In [11]:
col_to_drop = [col for col in test.columns if (col not in train.columns)&(col !='id')]
col_to_drop # theses columns cannot be used in training model since train data doesn't has these values

['roof_matl_Metal',
 'roof_matl_Roll',
 'exterior_1st_PreCast',
 'exterior_2nd_Other',
 'exterior_2nd_PreCast',
 'mas_vnr_type_CBlock',
 'heating_GasA',
 'electrical_None',
 'kitchen_qual_Po',
 'sale_type_VWD']

In [12]:
len(col_to_drop)

10

In [13]:
#create new columns in test data and assign values to be 0
for col in col_to_add:
    test[col] = 0
test[col_to_add].head()

,ms_subclass_150,ms_zoning_C (all),utilities_NoSeWa,neighborhood_GrnHill,neighborhood_Landmrk,condition_2_Feedr,condition_2_RRAe,condition_2_RRAn,condition_2_RRNn,roof_matl_Membran,...,heating_OthW,heating_Wall,heating_qc_Po,electrical_Mix,functional_Sal,functional_Sev,garage_qual_Fa,pool_qc_Fa,pool_qc_Gd,misc_feature_TenC
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#drop unnecessary columns
test.drop(columns=col_to_drop,inplace=True)

In [15]:
test.shape

(879, 274)

## Polynomial Features

In [16]:
#look at high correlation columns with sale price
train.corr()[np.abs(train.corr()['saleprice']) > 0.5]['saleprice']

overall_qual        0.803819
year_built          0.573575
year_remod/add      0.550429
mas_vnr_area        0.510505
total_bsmt_sf       0.665280
1st_flr_sf          0.647755
gr_liv_area         0.719374
full_bath           0.537959
totrms_abvgrd       0.510611
garage_cars         0.648897
garage_area         0.655706
saleprice           1.000000
exter_qual_TA      -0.602158
foundation_PConc    0.531886
kitchen_qual_TA    -0.540861
Name: saleprice, dtype: float64

In [17]:
#create polynomial columns deg =2, interaction only
poly_columns = [col for col in train.corr()[np.abs(train.corr()['saleprice']) > 0.5]['saleprice'].index if col != 'saleprice']
poly_columns

['overall_qual',
 'year_built',
 'year_remod/add',
 'mas_vnr_area',
 'total_bsmt_sf',
 '1st_flr_sf',
 'gr_liv_area',
 'full_bath',
 'totrms_abvgrd',
 'garage_cars',
 'garage_area',
 'exter_qual_TA',
 'foundation_PConc',
 'kitchen_qual_TA']

In [18]:
len(poly_columns)

14

In [19]:
#Instantiate PolynomialFeatures + create train_poly_df
poly = PolynomialFeatures(include_bias=False,degree=2,interaction_only=True)
train_poly = poly.fit_transform(train[poly_columns])
train_poly_df = pd.DataFrame(train_poly,columns=poly.get_feature_names(poly_columns))
train_poly_df.head()

,overall_qual,year_built,year_remod/add,mas_vnr_area,total_bsmt_sf,1st_flr_sf,gr_liv_area,full_bath,totrms_abvgrd,garage_cars,...,garage_cars garage_area,garage_cars exter_qual_TA,garage_cars foundation_PConc,garage_cars kitchen_qual_TA,garage_area exter_qual_TA,garage_area foundation_PConc,garage_area kitchen_qual_TA,exter_qual_TA foundation_PConc,exter_qual_TA kitchen_qual_TA,foundation_PConc kitchen_qual_TA
0,6.0,1976.0,2005.0,289.0,725.0,725.0,1479.0,2.0,6.0,2.0,...,950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.0,1996.0,1997.0,132.0,913.0,913.0,2122.0,2.0,8.0,2.0,...,1118.0,0.0,2.0,0.0,0.0,559.0,0.0,0.0,0.0,0.0
2,5.0,1953.0,2007.0,0.0,1057.0,1057.0,1057.0,1.0,5.0,1.0,...,246.0,1.0,0.0,0.0,246.0,0.0,0.0,0.0,0.0,0.0
3,5.0,2006.0,2007.0,0.0,384.0,744.0,1444.0,2.0,7.0,2.0,...,800.0,2.0,2.0,2.0,400.0,400.0,400.0,1.0,1.0,1.0
4,6.0,1900.0,1993.0,0.0,676.0,831.0,1445.0,2.0,6.0,2.0,...,968.0,2.0,2.0,2.0,484.0,484.0,484.0,1.0,1.0,1.0


In [20]:
#combine this to original data frame
for col in train_poly_df.columns:
    if col not in poly_columns:
        train[col] = train_poly_df[col]
train.shape #original #columns is 274 + #91 new columns (14/2*13) = 365 columns

(2045, 365)

In [21]:
#Repeat the same for test data
poly = PolynomialFeatures(include_bias=False,degree=2,interaction_only=True)
test_poly = poly.fit_transform(test[poly_columns])
test_poly_df = pd.DataFrame(test_poly,columns=poly.get_feature_names(poly_columns))
test_poly_df.head()

,overall_qual,year_built,year_remod/add,mas_vnr_area,total_bsmt_sf,1st_flr_sf,gr_liv_area,full_bath,totrms_abvgrd,garage_cars,...,garage_cars garage_area,garage_cars exter_qual_TA,garage_cars foundation_PConc,garage_cars kitchen_qual_TA,garage_area exter_qual_TA,garage_area foundation_PConc,garage_area kitchen_qual_TA,exter_qual_TA foundation_PConc,exter_qual_TA kitchen_qual_TA,foundation_PConc kitchen_qual_TA
0,6.0,1910.0,1950.0,0.0,1020.0,908.0,1928.0,2.0,9.0,1.0,...,440.0,1.0,0.0,0.0,440.0,0.0,0.0,0.0,0.0,0.0
1,5.0,1977.0,1977.0,0.0,1967.0,1967.0,1967.0,2.0,10.0,2.0,...,1160.0,2.0,0.0,2.0,580.0,0.0,580.0,0.0,1.0,0.0
2,7.0,2006.0,2006.0,0.0,654.0,664.0,1496.0,2.0,7.0,2.0,...,852.0,0.0,2.0,0.0,0.0,426.0,0.0,0.0,0.0,0.0
3,5.0,1923.0,2006.0,0.0,968.0,968.0,968.0,1.0,5.0,2.0,...,960.0,0.0,0.0,2.0,0.0,0.0,480.0,0.0,0.0,0.0
4,6.0,1963.0,1963.0,247.0,1394.0,1394.0,1394.0,1.0,6.0,2.0,...,1028.0,2.0,0.0,2.0,514.0,0.0,514.0,0.0,1.0,0.0


In [22]:
for col in test_poly_df.columns:
    if col not in poly_columns:
        test[col] = test_poly_df[col]
test.shape

(879, 365)

## write to csv

In [23]:
#write to csv
train.to_csv('../datasets/train_cleaned_preprocessed.csv',index=False)
test.to_csv('../datasets/test_cleaned_preprocessed.csv',index=False)

#move to next notebook: Project_02_03

In [24]:
test

,id,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,...,garage_cars garage_area,garage_cars exter_qual_TA,garage_cars foundation_PConc,garage_cars kitchen_qual_TA,garage_area exter_qual_TA,garage_area foundation_PConc,garage_area kitchen_qual_TA,exter_qual_TA foundation_PConc,exter_qual_TA kitchen_qual_TA,foundation_PConc kitchen_qual_TA
0,2658,69.0000,9142,6,8,1910,1950,0.0,0,0,...,440.0,1.0,0.0,0.0,440.0,0.0,0.0,0.0,0.0,0.0
1,2718,69.0552,9662,5,4,1977,1977,0.0,0,0,...,1160.0,2.0,0.0,2.0,580.0,0.0,580.0,0.0,1.0,0.0
2,2414,58.0000,17104,7,5,2006,2006,0.0,554,0,...,852.0,0.0,2.0,0.0,0.0,426.0,0.0,0.0,0.0,0.0
3,1989,60.0000,8520,5,6,1923,2006,0.0,0,0,...,960.0,0.0,0.0,2.0,0.0,0.0,480.0,0.0,0.0,0.0
4,625,69.0552,9500,6,5,1963,1963,247.0,609,0,...,1028.0,2.0,0.0,2.0,514.0,0.0,514.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1662,80.0000,8000,6,6,1974,1974,0.0,931,153,...,976.0,2.0,0.0,2.0,488.0,0.0,488.0,0.0,1.0,0.0
875,1234,90.0000,14670,6,7,1966,1999,410.0,575,0,...,960.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
876,1373,55.0000,8250,5,5,1968,1968,0.0,250,492,...,322.0,1.0,0.0,1.0,322.0,0.0,322.0,0.0,1.0,0.0
877,1672,60.0000,9000,4,6,1971,1971,0.0,616,0,...,1056.0,2.0,2.0,2.0,528.0,528.0,528.0,1.0,1.0,1.0
